In [1]:
import boto3
import sagemaker
from sagemaker.tensorflow import TensorFlow
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner
import json
print("boto3 : ", boto3.__version__)
print("sagemaker : ", sagemaker.__version__)

pandas failed to import. Analytics features will be impaired or broken.


boto3 :  1.14.44
sagemaker :  2.16.1


In [2]:
# トレーニングジョブの設定定義
# ここまでは単発のトレーニングジョブ実行時と同様

get_s3_loc = lambda *args: "s3://" + "/".join(args)

with open("./secrets.json", "r", encoding="utf-8") as fp:
    secrets = json.load(fp)
role=secrets["RoleArn"]
s3_bucket=secrets["S3Bucket"]

boto_session = boto3.Session(region_name="ap-northeast-1")
sess = sagemaker.Session(
    boto_session=boto_session,
    default_bucket=s3_bucket,
)

s3_train_loc = get_s3_loc(sess.default_bucket(), "train")
instance_types = {"CPU" : "ml.m5.large", "GPU" : "ml.g4dn.xlarge", "LOCAL" : "local"}

hyperparameters = {
    "EPOCH" : 50,
    "STEP" : 10,
    "MAX_IMAGE_SIZE" : 1024,
    "TB_BUCKET" : s3_train_loc,
    "MAX_TRIAL" : 9,
}

estimator = TensorFlow(
    entry_point='train.py',
    role=role,
    instance_count=1,
    instance_type=instance_types["GPU"],
    framework_version='2.3.0',
    py_version='py37',
    debugger_hook_config=False,
    hyperparameters=hyperparameters,
    sagemaker_session=sess,
    use_spot_instances=True,
    max_run=3600,
    max_wait=3600,
)

In [3]:
# チューニングするパラメータと、取り得るパラメータの範囲を定義
hyperparameter_ranges = {
    "CONTENT_WEIGHTS" : ContinuousParameter(
        min_value=5000, max_value=15000,
    ),
    "STYLE_WEIGHTS" : ContinuousParameter(
        min_value=0.001, max_value=0.1,
    ),
    "TOTAL_VARIATION_WEIGHTS" : ContinuousParameter(
        min_value=10, max_value=50,
    ),
    "LEARNING_RATE" : ContinuousParameter(
        min_value=0.01, max_value=0.1,
    ),
}

In [4]:
# モデルの性能を比較するための目標メトリクスを定義
# loss値が最小になるハイパーパラメータの組み合わせを今回は探ってみる
objective_metric_name = 'loss'
objective_type = 'Minimize'
metric_definitions = [
    {
        'Name': 'loss',
        # トレーニングスクリプト内でprint()されたメッセージは
        # CloudWatchLogsにログ出力される。
        # そのメッセージのうち、"FinalMeanLoss=([0-9\\.]+)"のパターンに合致する
        # 数値を目標メトリクスとして収集し、トレーニングジョブ間で比較する。
        'Regex': 'FinalMeanLoss=([0-9\\.]+)',
    }
]

In [9]:
# ハイパーパラメータチューニングジョブ設定を定義
tuner = HyperparameterTuner(
    # トレーニングジョブの定義
    estimator=estimator,
    # チューニング対象のハイパーパラメータ
    hyperparameter_ranges=hyperparameter_ranges,
    # 目標メトリクスの定義（今回はloss値を最小にする）
    objective_type=objective_type,
    objective_metric_name=objective_metric_name,
    metric_definitions=metric_definitions,
    # トータルで実行されるトレーニングジョブの上限
    max_jobs=30,
    # 並列実行されるジョブの上限
    # ml.g4dn.xlargeインスタンスのデファルトの同時起動上限数は２
    max_parallel_jobs=2,

)

In [10]:
# ハイパーパラメータチューニングの実行
tuner.fit(s3_train_loc, wait=False)

In [15]:
# 最善の結果を出したモデルの、ハイパーパラメータの組み合わせを確認する。
from IPython.display import display
best_hyperparameters = tuner.best_estimator().hyperparameters()
display(best_hyperparameters)


2020-11-18 17:34:11 Starting - Preparing the instances for training
2020-11-18 17:34:11 Downloading - Downloading input data
2020-11-18 17:34:11 Training - Training image download completed. Training in progress.
2020-11-18 17:34:11 Uploading - Uploading generated training model
2020-11-18 17:34:11 Completed - Training job completed


{'CONTENT_WEIGHTS': '5277.5099011496795',
 'EPOCH': '50',
 'LEARNING_RATE': '0.03725186282831269',
 'MAX_IMAGE_SIZE': '1024',
 'MAX_TRIAL': '9',
 'STEP': '10',
 'STYLE_WEIGHTS': '0.0012957731765036497',
 'TB_BUCKET': '"s3://sagemaker-nst/train"',
 'TOTAL_VARIATION_WEIGHTS': '10.0',
 '_tuning_objective_metric': '"loss"',
 'sagemaker_container_log_level': '20',
 'sagemaker_estimator_class_name': '"TensorFlow"',
 'sagemaker_estimator_module': '"sagemaker.tensorflow.estimator"',
 'sagemaker_job_name': '"tensorflow-training-2020-11-18-11-30-35-845"',
 'sagemaker_program': '"train.py"',
 'sagemaker_region': '"ap-northeast-1"',
 'sagemaker_submit_directory': '"s3://sagemaker-nst/tensorflow-training-2020-11-18-11-30-35-845/source/sourcedir.tar.gz"',
 'model_dir': '"s3://sagemaker-nst/tensorflow-training-2020-11-18-11-26-25-821/model"'}